In [24]:
from pyspark import (SparkConf, SparkContext)
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from tqdm import tqdm

import pyspark.sql.functions as F
import pyspark
import random
import pickle
import pandas as pd
import time

In [2]:
ss = SparkSession.builder.appName('testa').getOrCreate()

In [26]:
for i in tqdm(range(0, 79)): 
    with open('./pandaset_data/' + str(i) + '.pkl', 'rb') as p_f:
            data = pickle.load(p_f)

    refined_data = data.drop(['uuid', 'attributes.pedestrian_behavior', 'attributes.rider_status', 
                              'attributes.object_motion', 'attributes.pedestrian_age', 'cuboids.sibling_id', 
                              'cuboids.sensor_id', 'camera_used', 'stationary'], axis=1)
    
    if i == 0:
        df = pd.DataFrame(refined_data)
    else:
        df = pd.concat([df, pd.DataFrame(refined_data)], ignore_index=True)
    
print(df.info())
df.to_csv("pandaspark.csv")

100%|█████████████████████████████████████████████████████████████████████████████████| 79/79 [00:00<00:00, 431.71it/s]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11751 entries, 0 to 11750
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   label         11751 non-null  object 
 1   yaw           11751 non-null  float64
 2   position.x    11751 non-null  float64
 3   position.y    11751 non-null  float64
 4   position.z    11751 non-null  float64
 5   dimensions.x  11751 non-null  float64
 6   dimensions.y  11751 non-null  float64
 7   dimensions.z  11751 non-null  float64
dtypes: float64(7), object(1)
memory usage: 734.6+ KB
None


In [28]:
# dara = ss.read.option("header", "true").csv("pandaspark.csv")

schema = StructType([
    StructField('label', StringType(), True),
    StructField('yaw', FloatType(), True),
    StructField('position.x', FloatType(), True),
    StructField('position.y', FloatType(), True),
    StructField('position.z', FloatType(), True),
    StructField('dimensions.x', FloatType(), True),
    StructField('dimensions.y', FloatType(), True),
    StructField('dimensions.z', FloatType(), True)
])

ddf = ss.read.schema(schema = schema).csv("pandaspark.csv", header=True)
# ddf.show()
ddf.where((ddf.label == "Car") | (ddf.label == "Pedestrian")).show(truncate=False)

# ddff.show()
# print(df)

+----------+-----------+----------+----------+----------+------------+------------+------------+
|label     |yaw        |position.x|position.y|position.z|dimensions.x|dimensions.y|dimensions.z|
+----------+-----------+----------+----------+----------+------------+------------+------------+
|Car       |2.3803062  |20.759    |31.401    |0.591     |1.867       |4.629       |1.673       |
|Car       |2.3267868  |-60.458   |-37.423   |0.934     |2.104       |4.215       |1.681       |
|Car       |2.3803062  |26.281    |36.583    |0.451     |1.867       |4.689       |1.511       |
|Car       |-0.79735255|-44.205   |-44.985   |0.986     |2.049       |4.409       |1.733       |
|Car       |0.74077624 |-72.901   |100.37    |0.519     |1.825       |4.441       |1.641       |
|Car       |2.374344   |82.091    |107.072   |0.679     |1.99        |4.386       |1.76        |
|Car       |2.3984423  |-30.673   |-7.523    |0.759     |1.849       |4.723       |1.561       |
|Car       |-0.79162264|-10.89

In [ ]:
ddf.toPandas().to_csv('./structed_human_car_output.csv')